## import包

In [1]:
import sys
sys.path.append('../AutoX')

In [2]:
import datetime
import pandas as pd
import os
from autox.autox_recommend.recall_and_rank.feature_engineer import feature_engineer
from autox.autox_recommend.recall_and_rank.ranker import ranker, ranker_test, inference
from autox.autox_recommend.recall_and_rank.recalls import binary_recall
from autox.autox_recommend.recall_and_rank.recalls import history_recall
from autox.autox_recommend.recall_and_rank.recalls import itemcf_recall
from autox.autox_recommend.recall_and_rank.recalls import popular_recall
from autox.autox_recommend.recall_and_rank.recalls import w2v_concent_recall
from autox.autox_recommend.metrics import mapk

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not mo

In [3]:
# from autox import AutoXRecommend

In [4]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## 读取数据

In [5]:
path = '/home/caihengxing/kaggle/h_and_m/input/'

In [6]:
inter_df = pd.read_csv(path + 'transactions_train.csv', dtype={'article_id': str})
user_df = pd.read_csv(path + 'customers.csv')
item_df = pd.read_csv(path + 'articles.csv', dtype={'article_id': str})
sample_submission = pd.read_csv(path + 'sample_submission.csv')

In [7]:
inter_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [8]:
class AutoXRecommend():
    def __init__(self):
        pass

    def fit(self, inter_df, user_df, item_df,
                  uid, iid, time_col,
                  recall_num,
                  time_decay=0.8,
                  debug=False, debug_save_path=None):

        self.inter_df = inter_df
        self.user_df = user_df
        self.item_df = item_df

        self.uid = uid
        self.iid = iid
        self.time_col = time_col
        self.recall_num = recall_num
        self.time_decay = time_decay
        self.debug = debug

        if debug:
            assert debug_save_path is not None
            path_output = debug_save_path
            self.path_output = path_output
            os.makedirs(path_output, exist_ok=True)

        temp_date = datetime.datetime.strptime(str(inter_df[time_col].max()), '%Y-%m-%d %H:%M:%S') + \
                    datetime.timedelta(days=1)
        valid_date = str(datetime.datetime(temp_date.year, temp_date.month, temp_date.day))
        self.valid_date = valid_date

        train_date = datetime.datetime.strptime(valid_date, '%Y-%m-%d %H:%M:%S') - datetime.timedelta(days=7)
        train_date = str(train_date)


        print('\npopular_recall')
        print('train')
        popular_recall_train = popular_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('valid')
        popular_recall_valid = popular_recall(None, inter_df, date=valid_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')

        print('\nhistory_recall')
        print('train')
        history_recall_train = history_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('valid')
        history_recall_valid = history_recall(None, inter_df, date=valid_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')

        print('\nitemcf_recall')
        print('train')
        if os.path.exists(f'{path_output}/itemcf_recall_train.hdf'):
            itemcf_recall_train = pd.read_hdf(f'{path_output}/itemcf_recall_train.hdf')
        else:
            itemcf_recall_train = itemcf_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train',
                                                topk=1000, use_iif=False, sim_last_days=14,
                                                time_decay=time_decay)
        print('valid')
        if os.path.exists(f'{path_output}/itemcf_recall_valid.hdf'):
            itemcf_recall_valid = pd.read_hdf(f'{path_output}/itemcf_recall_valid.hdf')
        else:
            itemcf_recall_valid = itemcf_recall(None, inter_df, date=valid_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train',
                                                topk=1000, use_iif=False, sim_last_days=14,
                                                time_decay=time_decay)
        if debug:
            itemcf_recall_train.to_hdf(f'{path_output}/itemcf_recall_train.hdf', 'w', complib='blosc', complevel=5)
            itemcf_recall_valid.to_hdf(f'{path_output}/itemcf_recall_valid.hdf', 'w', complib='blosc', complevel=5)

        print('\nbinary_recall')
        print('train')
        if os.path.exists(f'{path_output}/binary_recall_train.hdf'):
            binary_recall_train = pd.read_hdf(f'{path_output}/binary_recall_train.hdf')
        else:
            binary_recall_train = binary_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train', topk=1000)

        print('valid')
        if os.path.exists(f'{path_output}/binary_recall_valid.hdf'):
            binary_recall_valid = pd.read_hdf(f'{path_output}/binary_recall_valid.hdf')
        else:
            binary_recall_valid = binary_recall(None, inter_df, date=valid_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train', topk=1000)
        if debug:
            binary_recall_train.to_hdf(f'{path_output}/binary_recall_train.hdf', 'w', complib='blosc', complevel=5)
            binary_recall_valid.to_hdf(f'{path_output}/binary_recall_valid.hdf', 'w', complib='blosc', complevel=5)

        print('\nw2v_content_recall')
        print('train')
        if os.path.exists(f'{path_output}/w2v_content_recall_train.hdf'):
            w2v_content_recall_train = pd.read_hdf(f'{path_output}/w2v_content_recall_train.hdf')
        else:
            w2v_content_recall_train = w2v_concent_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, dtype='train',
                                                topn = 20, topk = 20, prefix = 'w2v')

        print('valid')
        if os.path.exists(f'{path_output}/w2v_content_recall_valid.hdf'):
            w2v_content_recall_valid = pd.read_hdf(f'{path_output}/w2v_content_recall_valid.hdf')
        else:
            w2v_content_recall_valid = w2v_concent_recall(None, inter_df, date=valid_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, dtype='train',
                                                topn=20, topk=20, prefix='w2v')
        if debug:
            w2v_content_recall_train.to_hdf(f'{path_output}/w2v_content_recall_train.hdf', 'w', complib='blosc', complevel=5)
            w2v_content_recall_valid.to_hdf(f'{path_output}/w2v_content_recall_valid.hdf', 'w', complib='blosc', complevel=5)

        # 合并召回数据
        print('\nmerge recalls')
        print('train')
        history_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        itemcf_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        binary_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        train = popular_recall_train.append(history_recall_train)
        train.drop_duplicates(subset=[uid, iid], keep='first', inplace=True)
        train = train.merge(itemcf_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(binary_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(w2v_content_recall_train, on=[uid, iid, 'label'], how='outer')

        print('valid')
        history_recall_valid.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        itemcf_recall_valid.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        binary_recall_valid.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        valid = popular_recall_valid.append(history_recall_valid)
        valid.drop_duplicates(subset=[uid, iid], keep='first', inplace=True)
        valid = valid.merge(itemcf_recall_valid, on=[uid, iid, 'label'], how='outer')
        valid = valid.merge(binary_recall_valid, on=[uid, iid, 'label'], how='outer')
        valid = valid.merge(w2v_content_recall_valid, on=[uid, iid, 'label'], how='outer')

        # 特征工程
        print('\nfeature engineer')
        print('train')
        if os.path.exists(f'{path_output}/train_fe.hdf'):
            train_fe = pd.read_hdf(f'{path_output}/train_fe.hdf')
        else:
            train_fe = feature_engineer(train, inter_df,
                                        date=train_date,
                                        user_df=user_df, item_df=item_df,
                                        uid=uid, iid=iid, time_col=time_col,
                                        last_days=7, dtype='train')
        print('valid')
        if os.path.exists(f'{path_output}/valid_fe.hdf'):
            valid_fe = pd.read_hdf(f'{path_output}/valid_fe.hdf')
        else:
            valid_fe = feature_engineer(valid, inter_df,
                                        date=valid_date,
                                        user_df=user_df, item_df=item_df,
                                        uid=uid, iid=iid, time_col=time_col,
                                        last_days=7, dtype='train')

        if debug:
            train_fe.to_hdf(f'{path_output}/train_fe.hdf', 'w', complib='blosc', complevel=5)
            valid_fe.to_hdf(f'{path_output}/valid_fe.hdf', 'w', complib='blosc', complevel=5)


        iid2idx = {}
        idx2iid = {}
        for idx, cur_iid in enumerate(train_fe[iid].unique()):
            iid2idx[cur_iid] = idx
            idx2iid[idx] = cur_iid
        self.iid2idx = iid2idx
        train_fe[iid + '_idx'] = train_fe[iid].map(iid2idx)
        valid_fe[iid + '_idx'] = valid_fe[iid].map(iid2idx)

        print(f"train_fe shape: {train_fe.shape}")
        print(f"valid_fe shape: {valid_fe.shape}")

        print('\nranker')
        # todo: 检查train_fe中是否有冗余特征, 方差为0的特征

        lgb_ranker, valid_pred = ranker(train_fe, valid_fe,
                                        uid=uid, iid=iid, time_col=time_col)

        print('\nlocal result calculation')
        # 离线结果打印
        valid_pred = valid_pred.sort_values('prob', ascending=False)
        valid_pred = valid_pred.groupby(uid).head(12).groupby(uid)[iid].agg(list).reset_index()

        begin_date = datetime.datetime.strptime(valid_date, '%Y-%m-%d %H:%M:%S') - datetime.timedelta(days=7)
        begin_date = str(begin_date)

        valid_true = inter_df.loc[inter_df[uid].isin(valid_pred[uid])]
        valid_true = valid_true[(valid_true[time_col] <= valid_date) & (valid_true[time_col] > begin_date)]

        print(valid_true[time_col].min(), valid_true[time_col].max())
        valid_true = valid_true.groupby(uid)[iid].agg(list).reset_index()

        print("mAP Score on Validation set:", mapk(valid_true[iid], valid_pred[iid]))

        self.best_iteration_ = lgb_ranker.best_iteration_


        print("#" * 30)
        print('retrain')
        # 重新训练
        train_date = valid_date
        # train_date = '2022-04-07 00:00:00'

        print('\npopular_recall')
        popular_recall_train = popular_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('\nhistory_recall')
        history_recall_train = history_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('\nitemcf_recall')
        if os.path.exists(f'{path_output}/itemcf_recall_train_all.hdf'):
            itemcf_recall_train = pd.read_hdf(f'{path_output}/itemcf_recall_train_all.hdf')
        else:
            itemcf_recall_train = itemcf_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train',
                                                topk=1000, use_iif=False, sim_last_days=14,
                                                time_decay=time_decay)

        if debug:
            itemcf_recall_train.to_hdf(f'{path_output}/itemcf_recall_train_all.hdf', 'w', complib='blosc', complevel=5)

        print('\nbinary_recall')
        if os.path.exists(f'{path_output}/binary_recall_train_all.hdf'):
            binary_recall_train = pd.read_hdf(f'{path_output}/binary_recall_train_all.hdf')
        else:
            binary_recall_train = binary_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train', topk=1000)
        if debug:
            binary_recall_train.to_hdf(f'{path_output}/binary_recall_train_all.hdf', 'w', complib='blosc', complevel=5)

        print('\nw2v_content_recall')
        if os.path.exists(f'{path_output}/w2v_content_recall_train_all.hdf'):
            w2v_content_recall_train = pd.read_hdf(f'{path_output}/w2v_content_recall_train_all.hdf')
        else:
            w2v_content_recall_train = w2v_concent_recall(None, inter_df, date=train_date,
                                                          uid=uid, iid=iid, time_col=time_col,
                                                          last_days=7, dtype='train',
                                                          topn=20, topk=20, prefix='w2v')
        if debug:
            w2v_content_recall_train.to_hdf(f'{path_output}/w2v_content_recall_train_all.hdf', 'w', complib='blosc', complevel=5)


        # 合并召回数据
        print('\nmerge recalls')
        history_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        itemcf_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        binary_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        train = popular_recall_train.append(history_recall_train)
        train.drop_duplicates(subset=[uid, iid], keep='first', inplace=True)
        train = train.merge(itemcf_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(binary_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(w2v_content_recall_train, on=[uid, iid, 'label'], how='outer')

        # 特征工程
        print('\nfeature engineer')
        if os.path.exists(f'{path_output}/train_fe_all.hdf'):
            train_fe = pd.read_hdf(f'{path_output}/train_fe_all.hdf')
        else:
            train_fe = feature_engineer(train, inter_df,
                                        date=train_date,
                                        user_df=user_df, item_df=item_df,
                                        uid=uid, iid=iid, time_col=time_col,
                                        last_days=7, dtype='train')
        if debug:
            train_fe.to_hdf(f'{path_output}/train_fe_all.hdf', 'w', complib='blosc', complevel=5)


        train_fe[iid + '_idx'] = train_fe[iid].map(iid2idx)
        print(f"train_fe shape: {train_fe.shape}")

        print('\nranker')
        self.model, self.feats = ranker_test(train_fe, self.best_iteration_,
                                   uid=uid, iid=iid, time_col=time_col)


    def transform(self, uids):

        test_date = self.valid_date
        # test_date = '2022-04-07 00:00:00'

        print('\npopular recall, test')
        popular_recall_test = popular_recall(uids, self.inter_df, date=test_date,
                                             uid=self.uid, iid=self.iid, time_col=self.time_col,
                                             last_days=7, recall_num=self.recall_num, dtype='test')
        print('\nhistory recall, test')
        history_recall_test = history_recall(uids, self.inter_df, date=test_date,
                                             uid=self.uid, iid=self.iid, time_col=self.time_col,
                                             last_days=7, recall_num=self.recall_num, dtype='test')
        print('\nitemcf recall, test')
        if os.path.exists(f'{self.path_output}/itemcf_recall_test.hdf'):
            itemcf_recall_test = pd.read_hdf(f'{self.path_output}/itemcf_recall_test.hdf')
        else:
            itemcf_recall_test = itemcf_recall(uids, self.inter_df, date=test_date,
                                               uid=self.uid, iid=self.iid, time_col=self.time_col,
                                               last_days=7, recall_num=self.recall_num, dtype='test',
                                               topk=1000, use_iif=False, sim_last_days=14,
                                               time_decay=self.time_decay)
        if self.debug:
            itemcf_recall_test.to_hdf(f'{self.path_output}/itemcf_recall_test.hdf', 'w', complib='blosc', complevel=5)

        print('\nbinary recall, test')
        if os.path.exists(f'{self.path_output}/binary_recall_test.hdf'):
            binary_recall_test = pd.read_hdf(f'{self.path_output}/binary_recall_test.hdf')
        else:
            binary_recall_test = binary_recall(uids, self.inter_df, date=test_date,
                                               uid=self.uid, iid=self.iid, time_col=self.time_col,
                                               last_days=7, recall_num=self.recall_num, dtype='test', topk=1000)
        if self.debug:
            binary_recall_test.to_hdf(f'{self.path_output}/binary_recall_test.hdf', 'w', complib='blosc', complevel=5)

        print('\nw2v_content_recall, test')
        if os.path.exists(f'{self.path_output}/w2v_content_recall_test.hdf'):
            w2v_content_recall_test = pd.read_hdf(f'{self.path_output}/w2v_content_recall_test.hdf')
        else:
            w2v_content_recall_test = w2v_concent_recall(uids, self.inter_df, date=test_date,
                                                         uid=self.uid, iid=self.iid, time_col=self.time_col,
                                                         last_days=7, dtype='test',
                                                         topn=20, topk=20, prefix='w2v')
        if self.debug:
            w2v_content_recall_test.to_hdf(f'{self.path_output}/w2v_content_recall_test.hdf', 'w', complib='blosc', complevel=5)

        print('\nmerge recalls')
        history_recall_test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        itemcf_recall_test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        binary_recall_test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        test = popular_recall_test.append(history_recall_test)
        test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        test = test.merge(itemcf_recall_test, on=[self.uid, self.iid], how='outer')
        test = test.merge(binary_recall_test, on=[self.uid, self.iid], how='outer')
        test = test.merge(w2v_content_recall_test, on=[self.uid, self.iid], how='outer')

        print('\nfeature engineer')
        if os.path.exists(f'{self.path_output}/test_fe.hdf'):
            test_fe = pd.read_hdf(f'{self.path_output}/test_fe.hdf')
        else:
            test_fe = feature_engineer(test, self.inter_df,
                                       date=test_date,
                                       user_df=self.user_df, item_df=self.item_df,
                                       uid=self.uid, iid=self.iid, time_col=self.time_col,
                                       last_days=7, dtype='test')
        # if self.debug:
        #     test_fe.to_hdf(f'{self.path_output}/test_fe.hdf', 'w', complib='blosc', complevel=5)

        test_fe[self.iid + '_idx'] = test_fe[self.iid].map(self.iid2idx)
        print(f"test_fe shape: {test_fe.shape}")

        print('\ninference')
        bs = 60000
        recs = inference(self.model, self.feats, test_fe, uids,
                         uid=self.uid, iid=self.iid, time_col=self.time_col,
                         batch_size=bs)

        return recs


## 配置参数

In [9]:
uid = 'customer_id'
iid = 'article_id'
time_col = 't_dat'
recall_num = 50

## 注意时间列需要转成时间格式

In [10]:
inter_df[time_col] = pd.to_datetime(inter_df[time_col])

In [11]:
inter_df[time_col].min(), inter_df[time_col].max()

(Timestamp('2018-09-20 00:00:00'), Timestamp('2020-09-22 00:00:00'))

In [12]:
inter_df.shape

(31788324, 5)

In [13]:
import datetime
data_used_time = datetime.datetime.strptime('2020-06-22', '%Y-%m-%d')
inter_df = inter_df.loc[inter_df[time_col] > data_used_time]

In [14]:
inter_df[time_col].min(), inter_df[time_col].max()

(Timestamp('2020-06-23 00:00:00'), Timestamp('2020-09-22 00:00:00'))

In [15]:
inter_df.shape

(3981458, 5)

## 执行AutoXRecommend

In [16]:
autoXRecommend = AutoXRecommend()

autoXRecommend.fit(inter_df = inter_df, user_df = user_df, item_df = item_df,
                  uid = uid, iid = iid, time_col = time_col,
                  recall_num = recall_num,
                  debug = True, debug_save_path = './temp_h_and_m')


popular_recall
train
2020-09-10 00:00:00 2020-09-16 00:00:00


100%|██████████| 68613/68613 [00:03<00:00, 18469.01it/s]


HIT:  0.054450377315542246
valid
2020-09-17 00:00:00 2020-09-22 00:00:00


100%|██████████| 61950/61950 [00:03<00:00, 17919.77it/s]


HIT:  0.06762614152025388

history_recall
train
2020-09-10 00:00:00 2020-09-16 00:00:00


...


itemcf_recall
train
valid

binary_recall
train
valid

w2v_content_recall
train
valid

merge recalls
train
valid

feature engineer
train
valid
train_fe shape: (8704358, 62)
valid_fe shape: (7928507, 62)

ranker
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's map@12: 0.752447
[200]	valid_0's map@12: 0.753363
[300]	valid_0's map@12: 0.754015
[400]	valid_0's map@12: 0.754256
[500]	valid_0's map@12: 0.754468
[600]	valid_0's map@12: 0.754554
[700]	valid_0's map@12: 0.75459
[800]	valid_0's map@12: 0.754558
[900]	valid_0's map@12: 0.754766
[1000]	valid_0's map@12: 0.754916
[1100]	valid_0's map@12: 0.755056
[1200]	valid_0's map@12: 0.755124
[1300]	valid_0's map@12: 0.755181
[1400]	valid_0's map@12: 0.755171
[1500]	valid_0's map@12: 0.755278
[1600]	valid_0's map@12: 0.755244
Early stopping, best iteration is:
[1500]	valid_0's map@12: 0.755278
defaultdict(<class 'dict'>, {'valid_0': {'map@12': 0.7552783552988642}})
                        feature  importance
41    

100%|██████████| 61950/61950 [00:03<00:00, 17087.35it/s]


HIT:  0.06762614152025388

history_recall
2020-09-17 00:00:00 2020-09-22 00:00:00


100%|██████████| 44687/44687 [00:00<00:00, 65766.30it/s] 



itemcf_recall

binary_recall

w2v_content_recall

merge recalls

feature engineer
train_fe shape: (7927874, 62)

ranker
[100]	valid_0's map@12: 0.758154
[200]	valid_0's map@12: 0.758967
[300]	valid_0's map@12: 0.760189
[400]	valid_0's map@12: 0.760904
[500]	valid_0's map@12: 0.761908
[600]	valid_0's map@12: 0.762709
[700]	valid_0's map@12: 0.763572
[800]	valid_0's map@12: 0.764405
[900]	valid_0's map@12: 0.765207
[1000]	valid_0's map@12: 0.765899
[1100]	valid_0's map@12: 0.766596
[1200]	valid_0's map@12: 0.767253
[1300]	valid_0's map@12: 0.76785
[1400]	valid_0's map@12: 0.768497
[1500]	valid_0's map@12: 0.769044
defaultdict(<class 'dict'>, {'valid_0': {'map@12': 0.769044230098917}})
                        feature  importance
41               article_id_idx        4356
22                 product_code        3555
1                  binary_score        3419
32  purchase_corr_item_max_time        2801
40     latest_purchase_time_sub        2418
21                          age        2278

In [17]:
# mAP Score on Validation set: 0.02752116511050258


In [18]:
test_users = list(sample_submission[uid])
res = autoXRecommend.transform(test_users)


popular recall, test
2020-09-16 00:00:00 2020-09-22 00:00:00


100%|██████████| 1371980/1371980 [00:42<00:00, 32011.34it/s]



history recall, test

itemcf recall, test

binary recall, test


100%|██████████| 1371980/1371980 [4:50:16<00:00, 78.77it/s]   



w2v_content_recall, test


...

Begin training w2v model


   INFO -> PROGRESS: at sentence #90000, processed 680899 words, keeping 30224 word types
   INFO -> PROGRESS: at sentence #100000, processed 756450 words, keeping 30904 word types
   INFO -> PROGRESS: at sentence #110000, processed 831710 words, keeping 31607 word types
   INFO -> PROGRESS: at sentence #120000, processed 906131 words, keeping 32219 word types
   INFO -> PROGRESS: at sentence #130000, processed 980595 words, keeping 32739 word types
   INFO -> PROGRESS: at sentence #140000, processed 1054350 words, keeping 33297 word types
   INFO -> PROGRESS: at sentence #150000, processed 1128359 words, keeping 33813 word types
   INFO -> PROGRESS: at sentence #160000, processed 1204352 words, keeping 34271 word types
   INFO -> PROGRESS: at sentence #170000, processed 1279211 words, keeping 34726 word types
   INFO -> PROGRESS: at sentence #180000, processed 1353915 words, keeping 35114 word types
   INFO -> PROGRESS: at sentence #190000, processed 1427523 words, keeping 35506 word 

该循环程序运行时间： 21.37


100%|██████████| 530343/530343 [02:16<00:00, 3880.42it/s]


w2v content recall:  (9142540, 3)

merge recalls

feature engineer
customer feature engineer
num_cols: ['price', 'sales_channel_id']
cat_cols: []
interact feature engineer
test_fe shape: (124751319, 61)

inference
[1/23]
(5463386, 61)
[2/23]
(5449575, 61)
[3/23]
(5459309, 61)
[4/23]
(5449241, 61)
[5/23]
(5467410, 61)
[6/23]
(5450039, 61)
[7/23]
(5447103, 61)
[8/23]
(5462251, 61)
[9/23]
(5462474, 61)
[10/23]
(5452483, 61)
[11/23]
(5465537, 61)
[12/23]
(5450712, 61)
[13/23]
(5469452, 61)
[14/23]
(5449684, 61)
[15/23]
(5454679, 61)
[16/23]
(5444054, 61)
[17/23]
(5471083, 61)
[18/23]
(5445832, 61)
[19/23]
(5431956, 61)
[20/23]
(5454639, 61)
[21/23]
(5459955, 61)
[22/23]
(5457461, 61)
[23/23]
(4733004, 61)


In [19]:
res.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0568601043, 0779781015, 0801768001, 079112000..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0918522001, 0918292001, 0924243001, 075147100..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0794321007, 0794321011, 0805000007, 092424300..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0924243001, 0918522001, 0751471043, 075147100..."
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0896152002, 0924243001, 0924243002, 079158701..."


In [20]:
res['prediction'] = res['prediction'].apply(lambda x: ' '.join(x))

In [21]:
res.to_csv('./AutoX_hm_add_w2v.csv', index = False)

In [22]:
!zip -r AutoX_hm_add_w2v.csv.zip AutoX_hm_add_w2v.csv

  adding: AutoX_hm_add_w2v.csv (deflated 77%)


In [23]:
res.shape, sample_submission.shape

((1371980, 2), (1371980, 2))

In [24]:
res.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0779781015 0801768001 0791120002 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0918522001 0918292001 0924243001 0751471001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321011 0805000007 0924243001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0918522001 0751471043 0751471001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0924243001 0924243002 0791587015 07...


In [25]:
sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...
